# Using PYNQ library for PMOD_ADC

This just uses the built in Pmod_ADC library to read the value on the PMOD_AD2 peripheral.

In [20]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import Pmod_ADC
base = BaseOverlay("base.bit")

In [21]:
adc = Pmod_ADC(base.PMODA)

Read the raw value and the 12 bit values from channel 1.

Refer to docs: https://pynq.readthedocs.io/en/v2.1/pynq_package/pynq.lib/pynq.lib.pmod.html#pynq-lib-pmod

In [22]:
adc.read_raw(ch1=1, ch2=0, ch3=0)

[0]

In [37]:
help(adc)

Help on Pmod_ADC in module pynq.lib.pmod.pmod_adc object:

class Pmod_ADC(builtins.object)
 |  Pmod_ADC(mb_info)
 |  
 |  This class controls an Analog to Digital Converter Pmod.
 |  
 |  The Pmod AD2 (PB 200-217) is an analog-to-digital converter powered by 
 |  AD7991. Users may configure up to 4 conversion channels at 12 bits of 
 |  resolution.
 |  
 |  Attributes
 |  ----------
 |  microblaze : Pmod
 |      Microblaze processor instance used by this module.
 |  log_running : int
 |      The state of the log (0: stopped, 1: started).
 |  
 |  Methods defined here:
 |  
 |  __init__(self, mb_info)
 |      Return a new instance of an ADC object.
 |      
 |      Parameters
 |      ----------
 |      mb_info : dict
 |          A dictionary storing Microblaze information, such as the
 |          IP name and the reset name.
 |  
 |  get_log(self)
 |      Get the log of voltage values.
 |      
 |      First stop the log before getting the log.
 |      
 |      Returns
 |      -------
 |

In [23]:
adc.read(ch1=1, ch2=0, ch3=0)

[0.002]

# Using MicroblazeLibrary

Here we're going down a level and using the microblaze library to write I2C commands directly to the PMOD_AD2 peripheral

Use the documentation on the PMOD_AD2 to answer lab questions

In [24]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
base = BaseOverlay("base.bit")

In [25]:
liba = MicroblazeLibrary(base.PMODA, ['i2c'])

In [26]:
dir(liba) # list the available commands for the liba object

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_constants',
 '_build_functions',
 '_mb',
 '_populate_typedefs',
 '_rpc_stream',
 'active_functions',
 'i2c_close',
 'i2c_get_num_devices',
 'i2c_open',
 'i2c_open_device',
 'i2c_read',
 'i2c_write',
 'release',
 'reset',
 'visitor']

In the cell below, open a new i2c device. Check the resources for the i2c_open parameters

In [27]:
device = liba.i2c_open(3,2)# TODO open a device

In [28]:
dir(device) # list the commands for the device class

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_call_func',
 '_file',
 '_val',
 'close',
 'read',
 'write']

In [35]:
help(device)

Help on i2c in module pynq.lib.pynqmicroblaze.rpc object:

class i2c(builtins.object)
 |  i2c(val)
 |  
 |  Microblaze IIC Interface
 |  
 |  Implements low-level functions for read and writing to I2C slaves.
 |  The main comonents of the API are `i2c_read` and `i2c_write` which
 |  transfer a fixed number of bytes over the I2C bus. See the function
 |  documentation for more details.
 |  
 |  To create an I2C instace use `i2c_open` or `i2c_open_device`
 |  
 |  Methods defined here:
 |  
 |  __index__(self)
 |  
 |  __init__(self, val)
 |  
 |  __int__(self)
 |  
 |  __repr__(self)
 |  
 |  close = wrapped(self)
 |      Close an I2C Device
 |      
 |      `read` and `write` should not be called on the device once
 |      it is closed
 |  
 |  read = wrapped(self, slave_address, buffer, length)
 |      Read from the I2C Bus
 |      
 |      Reads a fixed number of bytes from the I2C bus
 |      
 |      Parameters
 |      ----------
 |      slave_address : int
 |          The address 

Below we write a command to the I2C channel and then read from the I2C channel. Change the buf[0] value to select different channels. See the AD spec sheet Configuration Register. https://www.analog.com/media/en/technical-documentation/data-sheets/AD7991_7995_7999.pdf

Changing the number of channels to read from will require a 2 byte read for each channel!

In [34]:
buf = bytearray(2)
buf[0] = int('00000000', 2)
device.write(0x28, buf, 1)
device.read(0x28, buf, 2)
print(format(int(((buf[0] << 8) | buf[1])), '#018b'))

0b0000000011101010


Compare the binary output given by ((buf[0]<<8) | buf[1]) to the AD7991 spec sheet. You can select the data only using the following command

In [30]:
result_12bit = (((buf[0] & 0x0F) << 8) | buf[1])

# Using MicroBlaze

In [17]:
base = BaseOverlay("base.bit")

In [18]:
%%microblaze base.PMODA

#include "i2c.h"

int read_adc(){
    i2c device = i2c_open(3, 2);
    unsigned char buf[2];
    buf[0] = 0;
    i2c_write(device, 0x28, buf, 1);
    i2c_read(device, 0x28, buf, 2);
    return ((buf[0] & 0x0F) << 8) | buf[1];
}

In [19]:
read_adc()

236